In [166]:
import os
import cv2
from PIL import Image
import subprocess
from PIL import Image, ImageDraw, ImageFont
import random

이부분은 flask랑 연동해서 각각 path 넣으면 될거 같아요

In [167]:
# 경로 설정
FACE_IMAGE_PATH = "C:/Users/SEHYUN02/Desktop/bback.jpg"  # 얼굴 이미지 경로
BACKGROUND_IMAGE_PATH ="D:/SimSwap/SimSwap-main/crop_224/6.jpg" # "C:/Users/SEHYUN02/Desktop/share.jpg"  # 배경 이미지 경로
CROP_SAVE_PATH = "D:/SimSwap/SimSwap-main/crop_224"  # 크롭된 이미지 저장 경로
DEEFAKE_RESULT_PATH = "D:/SimSwap/SimSwap-main/output/result.jpg"  # SimSwap 결과 경로 파일로 변경
FINAL_OUTPUT_PATH = "C:/Users/SEHYUN02/Desktop/final_output_image.jpg"  # 최종 합성 결과 저장 경로

CASCADE_PATH = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"  # Haar Cascade XML 경로

os.makedirs(CROP_SAVE_PATH, exist_ok=True)

In [168]:
def detect_and_crop_faces(image_path, crop_size=224, save_prefix=""):
    """
    이미지에서 모든 얼굴을 감지하고 crop_size x crop_size 크기로 크롭합니다.
    크롭된 이미지를 지정된 경로에 저장하고, 저장된 파일 경로와 좌표를 반환합니다.
    """
    # Haar Cascade 로드
    face_cascade = cv2.CascadeClassifier(CASCADE_PATH)

    # 컬러 이미지 읽기 (BGR 유지)
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # 얼굴 감지를 위해 그레이스케일 변환

    # 얼굴 감지
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        raise ValueError(f"No face detected in the image at {image_path}.")

    cropped_faces = []
    coords = []
    saved_paths = []

    # 원본 파일 이름 추출
    original_filename = os.path.splitext(os.path.basename(image_path))[0]

    # 각 얼굴에 대해 크롭 실행
    for i, (x, y, w, h) in enumerate(faces):
        center_x, center_y = x + w // 2, y + h // 2
        x1 = max(center_x - crop_size // 2, 0)
        y1 = max(center_y - crop_size // 2, 0)
        x2 = x1 + crop_size
        y2 = y1 + crop_size

        # 이미지 크기 초과 방지
        #height, width, _ = image.shape
        #if x2 > width or y2 > height:
            #continue

        cropped_face = image[y1:y2, x1:x2]  # 원본 컬러(BGR) 유지

        # 크롭된 이미지를 저장
        filename = f"{original_filename}_face_{i + 1}.jpg"
        save_path = os.path.join(CROP_SAVE_PATH, filename)
        cv2.imwrite(save_path, cropped_face)
        saved_paths.append(save_path)

        cropped_faces.append(cropped_face)
        coords.append((x1, y1, x2, y2))

    #if not cropped_faces:
        #raise ValueError(f"No face can be cropped with the given crop size ({crop_size}).")

    return saved_paths, coords

여기부터는 이제 서버에서 SIMSWAP 돌아가는걸로 대체하면 될거 같아요 </br>
우선은 os로 제 로컬에서 돌아가게 해놨는데, 각자 환경에 맞게 이부분만 변경하시면 됩니다

In [169]:
def run_simswap_model(face_path, background_path):
    """
    SimSwap 모델 실행. Anaconda Prompt에서 환경 활성화 및 디렉토리 변경 후 실행.
    """
    # 경로 정리
    face_path = os.path.normpath(face_path)
    background_path = os.path.normpath(background_path)
    simswap_dir = os.path.normpath("D:/SimSwap/SimSwap-main")

    # 명령어 구성
    commands = [
        "D:",  # D 드라이브로 이동
        "call conda activate simswap",  # Windows에서는 call 사용 필요
        f'cd {simswap_dir}',  # 작업 디렉토리 이동
        f'python test_one_image.py --crop_size 224 '
        f'--name people --Arc_path arcface_model/arcface_checkpoint.tar '
        f'--pic_a_path {face_path} --pic_b_path {background_path} --output_path output/'
    ]

    # 명령어를 &&로 연결하여 실행
    full_command = " && ".join(commands)

    try:
        subprocess.run(full_command, shell=True, check=True)
        print("SimSwap model executed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while running SimSwap: {e}")
        raise

In [170]:
def merge_deepfake_to_original(original_image_path, deepfake_image_path, coords, output_path):
    """
    딥페이크 결과를 배경 이미지의 지정된 좌표에 합성하여 저장.
    """
    # 배경 이미지와 딥페이크 결과 읽기
    original_image = Image.open(original_image_path)
    deepfake_image = Image.open(deepfake_image_path)

    # 배경 이미지에 딥페이크 이미지를 붙이기
    for (x1, y1, x2, y2) in coords:
        # 딥페이크 이미지를 크롭된 영역 크기에 맞게 리사이즈
        resized_deepfake = deepfake_image.resize((x2 - x1, y2 - y1))
        # 배경 이미지에 딥페이크 이미지를 붙이기
        original_image.paste(resized_deepfake, (x1, y1))

    # 최종 합성 이미지를 저장
    original_image.save(output_path)
    print(f"Final image saved to {output_path}")


In [171]:
def add_invisible_watermark(image_path, text_to_add, font_size=30, num_of_watermarks=10, font="arial.ttf"):
    """
    이미지에 비가시적 워터마크를 삽입 (원본 코드와 동일하게 동작).
    """

    # 입력 이미지 열기
    try:
        original_img = Image.open(image_path).convert("RGB")
    except FileNotFoundError as e:
        raise FileNotFoundError(f"Image file not found: {image_path}")

    # 이미지 크기
    width, height = original_img.size

    # 텍스트 템플릿 생성
    template_img = Image.new("RGB", original_img.size, (255, 255, 255))
    draw = ImageDraw.Draw(template_img)

    # 폰트 로드
    try:
        font_obj = ImageFont.truetype(font, font_size)
    except OSError:
        raise FileNotFoundError(f"Font file not found: {font}")

    # 텍스트 추가 (위치 계산 포함)
    y = 1
    direction = 0  # 0: 아래로, 1: 위로
    for _ in range(num_of_watermarks):
        if y > height:
            direction = 1
        elif y < 0:
            direction = 0

        y += random.randint(-50, 50) if direction == 1 else random.randint(50, 100)
        x = random.randint(0, max(0, width - 200))  # 텍스트 위치 계산
        draw.text((x, y), text_to_add, fill=(0, 0, 0), font=font_obj)

    # 워터마크 삽입 (LSB 방식)
    for row in range(height):
        for col in range(width):
            r, g, b = template_img.getpixel((col, row))
            oi_r, oi_g, oi_b = original_img.getpixel((col, row))

            if r < 128:  # 텍스트가 있는 경우
                if oi_r % 2 == 0:  # 픽셀을 홀수로 변경
                    oi_r = min(oi_r + 1, 255)
            else:  # 텍스트가 없는 경우
                if oi_r % 2 != 0:  # 픽셀을 짝수로 변경
                    oi_r = max(oi_r - 1, 0)

            original_img.putpixel((col, row), (oi_r, oi_g, oi_b))

    # 워터마크 삽입된 이미지 저장 (덮어쓰기)
    original_img.save(image_path)
    print(f"Invisible watermark added to {image_path}")


In [172]:
# 메인 실행 로직
if __name__ == "__main__":
    try:
        # 얼굴 감지 및 크롭 - 얼굴 이미지
        face_saved_paths, face_coords = detect_and_crop_faces(FACE_IMAGE_PATH)
        print(f"Detected and cropped faces from {FACE_IMAGE_PATH}: {face_saved_paths}")

        # 얼굴 감지 및 크롭 - 배경 이미지
        background_saved_paths, background_coords = detect_and_crop_faces(BACKGROUND_IMAGE_PATH)
        print(f"Detected and cropped backgrounds from {BACKGROUND_IMAGE_PATH}: {background_saved_paths}")

        # SimSwap 모델 실행
        if face_saved_paths and background_saved_paths:
            run_simswap_model(face_saved_paths[0], background_saved_paths[0])
        else:
            raise ValueError("No valid face or background images to process.")

        # 딥페이크 결과를 원본 이미지에 합성
        merge_deepfake_to_original(
            BACKGROUND_IMAGE_PATH,
            DEEFAKE_RESULT_PATH,
            background_coords,
            FINAL_OUTPUT_PATH
        )

        # 최종 이미지에 비가시적 워터마크 추가
        add_invisible_watermark(
            image_path=FINAL_OUTPUT_PATH,      # 최종 이미지 경로
            text_to_add="FAKE_ME", # 워터마크 텍스트
            font_size=30,                      # 텍스트 크기
            num_of_watermarks=10,              # 워터마크 개수
            font="arial.ttf"                   # 기본 폰트
        )

    except Exception as e:
        print(f"An error occurred: {str(e)}")

Detected and cropped faces from C:/Users/SEHYUN02/Desktop/bback.jpg: ['D:/SimSwap/SimSwap-main/crop_224\\bback_face_1.jpg']
Detected and cropped backgrounds from D:/SimSwap/SimSwap-main/crop_224/6.jpg: ['D:/SimSwap/SimSwap-main/crop_224\\6_face_1.jpg']
SimSwap model executed successfully.
Final image saved to C:/Users/SEHYUN02/Desktop/final_output_image.jpg
Invisible watermark added to C:/Users/SEHYUN02/Desktop/final_output_image.jpg


워터마크 추출 코드 (탐지 부분에 삽입 필요)

In [174]:
def check_extracted_watermark(output_path):
    """
    추출된 워터마크 파일을 시각적으로 확인.
    """
    from PIL import Image

    try:
        extracted_img = Image.open(output_path).convert("RGB")
    except FileNotFoundError as e:
        raise FileNotFoundError(f"Extracted watermark file not found: {output_path}")

    extracted_img.show()  # 추출된 워터마크 이미지 보기
    print(f"Extracted watermark checked: {output_path}")

In [173]:
def extract_invisible_watermark(image_path, output_path):
    """
    이미지에서 비가시적 워터마크를 추출하여 새로운 템플릿 이미지를 생성.
    """

    # 입력 이미지 열기
    try:
        watermarked_img = Image.open(image_path).convert("RGB")
    except FileNotFoundError as e:
        raise FileNotFoundError(f"Image file not found: {image_path}")

    # 원본 크기의 빈 이미지 생성 (워터마크 템플릿)
    width, height = watermarked_img.size
    extracted_template = Image.new("RGB", (width, height), (255, 255, 255))

    # 워터마크 추출
    for row in range(height):
        for col in range(width):
            r, g, b = watermarked_img.getpixel((col, row))

            if r % 2 == 0:  # 짝수는 텍스트 없음
                extracted_template.putpixel((col, row), (255, 255, 255))  # 흰색
            else:  # 홀수는 텍스트 있음
                extracted_template.putpixel((col, row), (0, 0, 0))  # 검정색

    # 추출된 템플릿 저장
    extracted_template.save(output_path)
    print(f"Watermark extracted and saved to {output_path}")

    return output_path  # 생성된 파일 경로 반환
